# GLORIA OOD

In [7]:
# imports
import numpy as np
from importlib import reload

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from datetime import datetime, time
import random 

import pandas
import seaborn as sns

from scipy.spatial import Delaunay

from oceancolor.hydrolight import loisel23
from oceancolor.insitu import gloria
from oceancolor.utils import spectra

In [2]:
%matplotlib ipympl

# Load up

## GLORIA

In [3]:
reload(gloria)
df_meta, df_Rrs, df_Rrs_std, df_qc_flags = gloria.load_gloria()

Loading GLORIA dataset...


In [4]:
gloria_wv, gloria_Rrs, _ = gloria.parse_table(df_Rrs, 'Rrs')
_, gloria_Rrs_std, _ = gloria.parse_table(df_Rrs_std, 'Rrs_std')

## L23

In [5]:
X,Y = 4,0
l23_ds = loisel23.load_ds(X, Y)

# Wavelengths, restricted to > 400 nm
#cut = (l23_ds.Lambda > min_wv) & (l23_ds.Lambda < high_cut)
l23_a = l23_ds.a.data[:]
l23_wave = l23_ds.Lambda.data

l23_Rs = l23_ds.Rrs.data[:]

# Rebin/cut GLORIA to L23

In [6]:
new_gloria_wv = np.append(l23_wave, [l23_wave.max()+5.]) - 2.5 # Because the rebinning is not interpolation
#
rwv_nm, gloria_rebin, gloria_rebin_sig = spectra.rebin_to_grid(gloria_wv, gloria_Rrs, gloria_Rrs_std, new_gloria_wv)

/home/xavier/Projects/Oceanography/python/ocean-color/oceancolor/utils/spectra.py:66: RuntimeWarning: invalid value encountered in divide
  isum = np.nansum(values[gd]*mask[gd], axis=0) / mask_sum
/home/xavier/Projects/Oceanography/python/ocean-color/oceancolor/utils/spectra.py:67: RuntimeWarning: invalid value encountered in divide
  esum = np.nansum(err_vals[gd]*mask[gd], axis=0) / mask_sum


In [8]:
l23_Rs.shape

(3320, 81)

# Delaunay Hull

## Build the hull

In [9]:
hull = Delaunay(l23_Rs)

QhullError: QH6235 qhull error (qh_memalloc): negative request size (-1211282984).  Did int overflow due to high-D?

While executing:  | qhull d Qc Qx Qt Qbb Qz Q12
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 636390655  delaunay  Qcoplanar-keep  Qxact-merge  Qtriangulate
  Qbbound-last  Qz-infinity-point  Q12-allow-wide  _zero-centrum
  Qinterior-keep  Q3-no-merge-vertices-dim-high  Pgood  _max-width 0.029
  Error-roundoff 4.9e-15  _one-merge 8.1e-13  Visible-distance 2.9e-14
  U-max-coplanar 2.9e-14  Width-outside 5.9e-14  _wide-facet 1.8e-13
  _maxoutside 8.1e-13
Last point added to hull was p2117.

At error exit:

Delaunay triangulation by the convex hull of 3321 points in 82-d:

  Number of input sites and at-infinity: 87
  Total number of nearly incident points: 3234
  Number of Delaunay regions: 2479008

Statistics for:  | qhull d Qc Qx Qt Qbb Qz Q12

  Number of points processed: 86
  Number of hyperplanes created: 293439
  Number of facets in hull: 2526988
  Number